In [81]:
from azureml.core import Workspace, Dataset

subscription_id = '20d4fdf3-6a4b-4f0b-a842-bd7392136332'
resource_group = 'cienciadatos'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_id(workspace, '9227d1d4-d528-48db-84ed-768975adbf63')
dataset.download(target_path='.', overwrite=False)

['c:\\Users\\pablo.tamayo\\Desktop\\DataSchool\\Azure\\AzureDataSchool\\train_data.csv']

In [ ]:
##%%writefile experiments/mlflow_preprocess.py
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
impoer mlflow
import numpy as np
import pandas as pd

df = pd.read_csv("../airlines_delay/airlines_delay.csv", sep = ",")

def clean_data(data:pd.DataFrame) -> pd.DataFrame:
    '''
    Delete the Fligh column
    input: dataset
    output: pandas dataframe withiout Flight
    '''
    clean_df = data.drop('Flight', axis=1, inplace=False)
    return clean_df


def preprocess_data(data:pd.DataFrame) -> pd.DataFrame:
    '''
    Preprocess the data converting the categorical columns to one hot encodig
    and apply standarScaler to numeric columns
    input: pandas dataFrame
    output: tuple, first element corrempond to pd.Dataframe of independent features
            second element is the dependent(target) feature 
    '''

    numeric_columns = ['Length','Time']
    category_columns = ['Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek']

    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                            ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))])

    preprocessor = ColumnTransformer(
                        transformers=[
                            ('numeric', numeric_transformer, numeric_columns),
                            ('cat', categorical_transformer, category_columns)
                        ],
                        remainder='passthrough'
                    )
    
    #split the dependent(y) and independent(df_x) features 
    df_X = data.drop('Class', axis=1, inplace=False)
    #y_labels = data['Class']
    y = data['Class']
    preprocessed_data = preprocessor.fit_transform(df_X)
    encoded_category = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(category_columns)

    labels = np.concatenate([numeric_columns, encoded_category])
    
    preprocessed_df = pd.DataFrame(data=preprocessed_data, columns=labels)
    return pd.concat([preprocessed_df, y], axis=1)



df_new = clean_data(df)
processed_df= preprocess_data(df_new)
processed_df.head()

#df_new.head()

In [ ]:
#%%writefile experiments/mlflow_eda.py

from funct.myfuncs import cln_data
import argparse

experiment_name ='exp-Airlines'
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    
    #loading data 
    #dataset = Dataset.get_by_name(ws, name='AirlinesDelay')
    df = pd.read_csv("../airlines_delay/airlines_delay.csv", sep = ",")
    
    #Distribution of the target column
    class_cero, class_one = df['Class'].value_counts()
    mlflow.log_metrics({'one':class_one, 'cero':class_cero})
    
    #Comparation of the mean
    mean_comparison = df[['Time','Length', 'Class']].groupby('Class').mean().to_dict()
 
    mlflow.log_metrics({'Mean Time class cero':mean_comparison['Time'][0], 'Mean Time class one':mean_comparison['Time'][1]})
    mlflow.log_metrics({'Mean Lenght class cero':mean_comparison['Length'][0], 'Mean Length class one':mean_comparison['Length'][1]})

    #Number of airlines
    number_of_airlines = len(df['Airline'].unique())

    mlflow.log_metric('number of airlines', number_of_airlines)

    #flights per airlnes
    flights_per_airline = df['Airline'].value_counts().to_dict()
    mlflow.log_metrics(flights_per_airline)

    
    #airports
    number_of_airports = len(df['AirportFrom'].unique())
    mlflow.log_metric('number of airports', number_of_airports)

    #airports with more activity
    arports_activity_df = df['AirportFrom'].value_counts().sort_values(ascending=False) + df['AirportTo'].value_counts().sort_values(ascending=False)
    top_5_airpots_activity = arports_activity_df.sort_values(ascending=False).head(5).to_dict()

    top_and_lowest_ariports_activity = {}
    for k, _ in top_5_airpots_activity.items():
        top_and_lowest_ariports_activity[f'Airport {k}'] = top_5_airpots_activity[k]


    #airports with more activity
    lowest_5_airpots_activity = arports_activity_df.sort_values(ascending=False).tail(5).to_dict()
    for k, _ in lowest_5_airpots_activity.items():
        top_and_lowest_ariports_activity[f'Airport {k}'] = lowest_5_airpots_activity[k]

    mlflow.log_metrics(top_and_lowest_ariports_activity)

    #DaysWeek
    dict_day_of_week= {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7: 'Sunday'}
    df_day_of_week = df.replace({"DayOfWeek": dict_day_of_week},inplace=False)
    fligths_per_day_of_week = df_day_of_week['DayOfWeek'].value_counts().to_dict()

    mlflow.log_metrics(fligths_per_day_of_week)


    #plot distribution of time
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()    
    plt.hist(df['Length'])
    plt.show()
    mlflow.log_figure(fig, "plot.png")
    print("Plot listo!")

In [1]:
import pandas as pd

df = pd.read_csv('train_data.csv', delimiter=',')


In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from azureml.core import Run
import pandas as pd
import numpy as np
import argparse
import os
import logging

x = df.drop('Class', axis=1, inplace=False)
y = df['Class']

'''
lr_cv = cross_validate(LogisticRegression(max_iter=400), x, y, cv=2, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo regresion lineal')
#rf_cv = cross_validate(RandomForestClassifier(), x, y, cv=3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
#print('Listo random forest')
svm = cross_validate(SVC(), x, y, cv = 3, scoring=['balanced_accuracy', 'recall', 'precision'], return_train_score=True)
print('Listo SVM')

hola =[]
for score_model, name_model in [(lr_cv, 'lr'), (svm, 'svm')]:
    hola.append({
        f'train_acc_{name_model}': np.mean(score_model['train_balanced_accuracy']),
        f'train_recall_{name_model}': np.mean(score_model['train_recall']),
        f'train_precision_{name_model}': np.mean(score_model['train_precision']),
        f'test_acc_{name_model}': np.mean(score_model['test_balanced_accuracy']),
        f'test_recall_{name_model}': np.mean(score_model['test_recall']),
        f'test_precision_{name_model}': np.mean(score_model['test_precision'])
    })


'''

param_dist ={
    'penalty':['l1', 'l2', 'elasticnet'],
    'C': [c/10 for c in range(1,20,5)],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

}
clf = RandomizedSearchCV(LogisticRegression(), param_dist, n_iter=5, random_state=0, refit='balanced_accuracy')
'''
mlflow.log_params({
    'penalty': 'l2',
    'C': 1.0,
    'solver':'lbfgs'
})
'''
clf.fit(x,y)

#mlflow.sklearn.log_model(lr, artifact_path='skleran-model')
y_pred = clf.predict(x)

recall = recall_score(y,y_pred)
precision = precision_score(y, y_pred)
acc = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
print(recall)
print(precision)
print(acc)
print(conf_matrix)

#logging.info(f'matriz de confusion{conf_matrix}')

'''
mlflow.log_metrics({
    'recall':recall,
    'precision': precision,
    'accuracy': acc,

})
'''

In [12]:
clf.best_params_

{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.6}

In [2]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from azureml.core import Run
import pandas as pd
import numpy as np
import argparse
import os
import logging

x = df.drop('Class', axis=1, inplace=False)
y = df['Class']


svm = SVC()

svm.fit(x,y)

y_pred = svm.predict(x)

recall = recall_score(y,y_pred)
precision = precision_score(y, y_pred)
acc = accuracy_score(y, y_pred)
conf_matrix = confusion_matrix(y, y_pred)
print(recall)
print(precision)
print(acc)
print(conf_matrix)